In [4]:
specification = { "specifications": "ballpoint pen, black, metal", "quantity": 30, "price": 5, "num_days": 7, "need_logo": 'false' }

In [5]:
def gpt_draft_mail(specification):
    # Create a GPT prompt
    pormpt = f"Write a mail to a vendor asking for a quotation for the following specifications:\nSpecifications: {specification['specifications']}\nQuantity: {specification['quantity']}\nPrice: {specification['price']}\nNumber of days: {specification['num_days']}\nNeed logo: {specification['need_logo']}\n\nMail:"

    return pormpt

In [ ]:
function_json = {
  "name": "get_specifications",
  "parameters": {
    "type": "object",
    "properties": {
    "requirements_satisfied": {
        "type": "boolean",
        "description": "Can all the requirements be satisfied by the vendor?"
      },
    "unfullfiled_requirements": {
            "type": "string",
            "description": "What requirements cannot be satisfied by the vendor?"
        },
      "quotation": {
        "type": "number",
        "description": "Price per unit of the product. Set to 0 if the requirements cannot be satisfied at all."
      },
      "procurement_days": {
        "type": "number",
        "description": "Number of days to fullfil the requirements. Set to 0 if the requirements cannot be satisfied at all."
      }
    },
    "required": [
      "requirements_satisfied",
      "unfullfiled_requirements",
      "quotation",
      "procurement_days"
    ]
  },
  "description": "Check if the vendor can fullfil the requirements and get the vendor quotation and procurement days from the mail.",
}

In [ ]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="Procurement Bot",
    instructions="You are an AI assistant that is supposed to write a mail to the vendor asking for a quotation. You are supposed to collect the 3 main information: Can the requirements be met, what is the price and how long will it take to deliver (in days). ",
    tools=[{"type": "function", "function": function_json, "name": "get_specifications"}],
    model="gpt-4-1106-preview",
)

output = interpreter_assistant.invoke({"content": "I need a hoodie for men."})
output

In [6]:
gpt_draft_mail(specification)

'Write a mail to a vendor asking for a quotation for the following specifications:\nSpecifications: ballpoint pen, black, metal\nQuantity: 30\nPrice: 5\nNumber of days: 7\nNeed logo: false\n\nMail:'